In [1]:
import os
import requests
import zipfile
import io
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

def get_nemweb_dispatch_info(start_date, end_date):
    # Convert input date strings to datetime objects
    print(f'Start Date: {start_date}, End Date: {end_date}')
    start_dt = datetime.strptime(start_date, "%d/%m/%Y %H:%M")
    og_start_dt = datetime.strptime(start_date, "%d/%m/%Y %H:%M")

    end_dt = datetime.strptime(end_date, "%d/%m/%Y %H:%M")
    og_end_dt = datetime.strptime(end_date, "%d/%m/%Y %H:%M")
    print(f'Start Date before formatting: {start_dt}, End Date before formatting: {end_dt}')

    print(f'Original Start date {og_start_dt}, Original End date {og_end_dt}')
    
    # Adjust start_dt if the time is before 04:05:00
    if start_dt.time() < datetime.strptime("04:05:00", "%H:%M:%S").time():
        start_dt -= timedelta(days=1)
        print(f'Start date after adjustment: {start_dt}')
    
    # Adjust end_dt if the time is after 04:05:00
    if end_dt.time() > datetime.strptime("04:05:00", "%H:%M:%S").time():
        end_dt += timedelta(days=1)
        print(f'End date after adjustment: {end_dt}')
    
    # Initialize an empty DataFrame to store the concatenated data
    combined_df = pd.DataFrame()
    
    # Base URL where files are stored
    base_url = "https://nemweb.com.au/Reports/CURRENT/Next_Day_Dispatch/"

    first_df = True

    # Iterate through each day between start_date and end_date
    while start_dt <= end_dt:
        # Format the date as YYYYMMDD for the file name
        formatted_date = start_dt.strftime("%Y%m%d")
        
        try:
            # Get the page content
            response = requests.get(base_url)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find all links in the page that match the date pattern
            file_links = soup.find_all('a', href=True)
            matching_files = [
                os.path.basename(link['href']) for link in file_links
                if formatted_date in link['href'] and link['href'].endswith('.zip')
            ]
            
            # Download and process each matching file
            for file_name in matching_files:
                file_url = base_url + file_name
                print(f'Downloading {file_url}')
                file_response = requests.get(file_url)
                file_response.raise_for_status()
                
                with zipfile.ZipFile(io.BytesIO(file_response.content)) as the_zip:
                    for file in the_zip.namelist():
                        with the_zip.open(file) as the_file:
                            if first_df:
                                # Read the first DataFrame without skipping rows
                                df = pd.read_csv(the_file, skiprows=1)
                                column_names = df.columns
                                first_df = False
                            else:
                                # Skip the first 2 rows for subsequent DataFrames
                                df = pd.read_csv(the_file, skiprows=2, names=column_names)
                                
                            df = df[df['UNIT_SOLUTION'] == 'UNIT_SOLUTION']
                            combined_df = pd.concat([combined_df, df], ignore_index=True, axis=0)
        
        except requests.exceptions.RequestException as e:
            print(f"Failed to process {formatted_date}: {e}")
        
        # Increment the date by one day
        start_dt += timedelta(days=1)

    
    columns_to_keep = ['SETTLEMENTDATE', 'DUID', 'INTERVENTION', 'DISPATCHMODE', 'AGCSTATUS', 'INITIALMW', 
                    'TOTALCLEARED', 'RAMPDOWNRATE', 'RAMPUPRATE', 'LOWER5MIN', 'LOWER60SEC', 
                    'LOWER6SEC', 'RAISE5MIN', 'RAISE60SEC', 'RAISE6SEC', 'LOWERREG', 'RAISEREG', 
                    'AVAILABILITY', 'RAISEREGENABLEMENTMAX', 'RAISEREGENABLEMENTMIN', 
                    'LOWERREGENABLEMENTMAX', 'LOWERREGENABLEMENTMIN', 'SEMIDISPATCHCAP']

    if 'LOWER1SEC' in combined_df.columns:
        columns_to_keep.append('LOWER1SEC')
        if combined_df['LOWER1SEC'].isnull().any():
            combined_df['LOWER1SEC'] = combined_df['LOWER1SEC'].fillna(0)
    
    if 'RAISE1SEC' in combined_df.columns:
        columns_to_keep.append('RAISE1SEC')
        if combined_df['RAISE1SEC'].isnull().any():
            combined_df['RAISE1SEC'] = combined_df['RAISE1SEC'].fillna(0)

    # Select only the columns you want to keep
    combined_df = combined_df.loc[:, columns_to_keep]

    combined_df['SETTLEMENTDATE'] = pd.to_datetime(combined_df['SETTLEMENTDATE'])
    combined_df = combined_df[(combined_df['SETTLEMENTDATE'] >= og_start_dt) & (combined_df['SETTLEMENTDATE'] <= og_end_dt)]
    
    return combined_df

In [2]:
df = get_nemweb_dispatch_info("01/09/2024 00:05", "18/09/2024 00:00")
output_file = f'dispatch_load_202409.parquet'
    
import pyarrow.parquet as pq

df.to_parquet(output_file, engine='fastparquet')

Start Date: 01/09/2024 00:05, End Date: 18/09/2024 00:00
Start Date before formatting: 2024-09-01 00:05:00, End Date before formatting: 2024-09-18 00:00:00
Original Start date 2024-09-01 00:05:00, Original End date 2024-09-18 00:00:00
Start date after adjustment: 2024-08-31 00:05:00


C:\Users\siddh\AppData\Local\Temp\ipykernel_37268\2785140603.py:72: DtypeWarning: Columns (5,7,8,9,10,11,12,13,14,15,16,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(the_file, skiprows=1)


C:\Users\siddh\AppData\Local\Temp\ipykernel_37268\2785140603.py:77: DtypeWarning: Columns (5,7,8,9,10,11,12,13,14,15,16,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(the_file, skiprows=2, names=column_names)


C:\Users\siddh\AppData\Local\Temp\ipykernel_37268\2785140603.py:77: DtypeWarning: Columns (5,7,8,9,10,11,12,13,14,15,16,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(the_file, skiprows=2, names=column_names)


C:\Users\siddh\AppData\Local\Temp\ipykernel_37268\2785140603.py:77: DtypeWarning: Columns (5,7,8,9,10,11,12,13,14,15,16,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(the_file, skiprows=2, names=column_names)


...

C:\Users\siddh\AppData\Local\Temp\ipykernel_37268\2785140603.py:77: DtypeWarning: Columns (5,7,8,9,10,11,12,13,14,15,16,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(the_file, skiprows=2, names=column_names)


C:\Users\siddh\AppData\Local\Temp\ipykernel_37268\2785140603.py:77: DtypeWarning: Columns (5,7,8,9,10,11,12,13,14,15,16,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(the_file, skiprows=2, names=column_names)


C:\Users\siddh\AppData\Local\Temp\ipykernel_37268\2785140603.py:77: DtypeWarning: Columns (5,7,8,9,10,11,12,13,14,15,16,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(the_file, skiprows=2, names=column_names)


C:\Users\siddh\AppData\Local\Temp\ipykernel_37268\2785140603.py:77: DtypeWarning: Columns (5,7,8,9,10,11,12,13,14,15,16,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(the_file, skiprows=2, names=column_names)


             SETTLEMENTDATE     DUID INTERVENTION DISPATCHMODE AGCSTATUS  INITIALMW TOTALCLEARED RAMPDOWNRATE RAMPUPRATE  LOWER5MIN  LOWER60SEC  LOWER6SEC  RAISE5MIN  RAISE60SEC  RAISE6SEC  LOWERREG  RAISEREG  AVAILABILITY  RAISEREGENABLEMENTMAX  RAISEREGENABLEMENTMIN  LOWERREGENABLEMENTMAX  LOWERREGENABLEMENTMIN  SEMIDISPATCHCAP  LOWER1SEC  RAISE1SEC
119280  2024-09-01 00:05:00  ADPBA1G            0            0         1        0.0          0.0        93.12      93.12        0.0         0.0        0.0        3.0         3.0        3.0       0.0       0.0       6.00000                6.00000                    0.0                6.00000                    0.0              0.0        0.0        0.0
119281  2024-09-01 00:05:00  ADPBA1L            0            0         1      0.266          1.0        93.12      93.12        3.0         3.0        3.0        0.0         0.0        0.0       0.0       0.0       6.00000                6.00000                    0.0                6.00000 

In [8]:
import pandas as pd

df = pd.read_parquet('../data/dispatch_price/dispatch_price_202301.parquet')
print(df)

           SETTLEMENTDATE REGIONID  INTERVENTION         RRP  RAISE6SECRRP  RAISE60SECRRP  RAISE5MINRRP  RAISEREGRRP  LOWER6SECRRP  LOWER60SECRRP  LOWER5MINRRP  LOWERREGRRP PRICE_STATUS  RAISE1SECRRP  LOWER1SECRRP         LASTCHANGED       DOWNLOAD_TIME
index                                                                                                                                                                                                                                                        
5     2023-01-01 00:05:00     NSW1             0  140.399551          0.39           0.86          0.50         5.45          0.39           0.86          0.39         3.70         FIRM             0             0 2023-01-01 00:30:02 2024-05-09 18:06:00
6     2023-01-01 00:05:00     QLD1             0  139.730164          0.39           0.86          0.50         5.45          0.39           0.86          0.39         3.70         FIRM             0             0 2023-01-01 00:30:02 2024